In [1]:
import numpy as np
import sys
import os
import shutil
import time
import cv2
from PIL import Image
from tqdm import tqdm
from matplotlib import pyplot as plt
import SimpleITK as sitk

from skimage import io
from skimage.measure import block_reduce
import dask_image.imread

from scipy import ndimage # For the non-integer (continuous) zoom function

from skelly_module import *

In [2]:
def get_dir(path):
    tiffs = [os.path.join(path, f) for f in os.listdir(path) if f[0] != '.']
    return sorted(tiffs)

In [7]:
""" 
# Fast but less precise

input_folder = r'/data/Lucas/AL210/AL210_1'
output_folder = r'/home/lucasdelez/Documents/AllenBrainCCFv3'

def downscale_imgs_allen(in_folder, out_folder, factor):
    img_stack = dask_image.imread.imread(in_folder + "/*.tif").compute()
    return block_reduce(img_stack, block_size=factor, func=np.mean)

img_stack = downscale_imgs_allen(input_folder, output_folder, factor=(5,5,5)) #Factor defined for Mesospim
write_tiff_stack(img_stack, r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al210_25um_corrected.tif')
"""

In [4]:
# Slow but non-integer subsampling possible

input_folder = r'/data/Lucas/AL211/AL211_1'
output_folder = r'/home/lucasdelez/Documents/AllenBrainCCFv3'

def downscale_imgs_allen(in_folder, out_folder, factor):
    img_stack = dask_image.imread.imread(in_folder + "/*.tif").compute()
    return ndimage.zoom(img_stack, zoom=factor)

zoom_seq = (5/25,5.3/25,5.3/25)
img_stack = downscale_imgs_allen(input_folder, output_folder, factor=zoom_seq) #Factor defined for Mesospim
write_tiff_stack(img_stack, r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al211_25um_corrected.tif')

### Some other stuff

In [7]:
write_tiff_stack(img_stack, r'/home/lucasdelez/Documents/AllenBrainCCFv3/Al211_25um.tif')

In [45]:
def downscale_imgs(in_folder, out_folder, factor=(10,10,1)):
    images = get_dir(in_folder)
    
    for _,item in enumerate(tqdm(images)):
        img = cv2.imread(item, -1) #black/white
        out_img = block_reduce(img, block_size=factor, func=np.mean)
        out_img = Image.fromarray(out_img)
        out_img.save(out_folder + "//" + "ds_" + os.path.basename(item))

In [46]:
input_folder = r'/data/Lucas/Substacks/AL215s/Al215-substack'
output_folder = r'/data/Lucas/Substacks/AL215s/Al215-substack-ds'

In [47]:
downscale_imgs(input_folder, output_folder, (10,10))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 62.11it/s]


In [2]:
input_folder = r"M:\data\AL177\AL177_COLM\RES(16214x17088x2335)"
output_folder = r"E:\Lucas\output folder"
downsampling_factor = 10

def downscale_imgs(in_folder, out_folder, factor):
    images=os.listdir(in_folder)
    images=[i for i in images if i.endswith('.tif')]
    images.sort()
    
    for count,item in enumerate(tqdm(images)):
        if count > 2335:
            img = cv2.imread(in_folder + "/" + item, -1)
            out_img = block_reduce(img, block_size=(factor, factor), func=np.mean)
            out_img = Image.fromarray(out_img)
            out_img.save(out_folder + "//" + "ds_" + os.path.basename(item))

downscale_imgs(input_folder, output_folder, downsampling_factor)    

100%|█████████████████████████████████████████████████████████████████████████| 2335/2335 [00:00<00:00, 2327400.15it/s]


In [3]:
tiff_stack_file = r"M:\data\AL177\177_561nm.tif"

#Can be extremely RAM intensive
image_stack = io.imread(tiff_stack_file)
print(image_stack.shape)

(1837, 2048, 2048)


In [4]:
selected_img = image_stack[800]

In [5]:
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) **2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

In [9]:
images=os.listdir(output_folder)
images=[i for i in images if i.endswith('.tif')]
images.sort()
dsize = selected_img.shape

img = cv2.imread(output_folder + "/" + images[0], -1)
img = cv2.resize(img, dsize) 
min_err = mse(selected_img, img)
best_comp = 0

for count,item in enumerate(tqdm(images)):
    img = cv2.imread(output_folder + "/" + item, -1)
    img = cv2.resize(img, dsize)
    err = mse(np.multiply(selected_img,1/np.max(selected_img)), np.multiply(img,1/np.max(img)))
    if min_err > err:
        min_err = err
        best_comp = count

100%|██████████████████████████████████████████████████████████████████████████████| 2335/2335 [06:07<00:00,  6.35it/s]


In [12]:
min_err

0.00018329006343443312

In [13]:
images[best_comp]

'ds_000000_000000_039600.tif'